In [ ]:
import pandas as pd
from ebaysdk.finding import Connection as Finding
from ebaysdk.exception import ConnectionError
pd.set_option('max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
APPLICATION_ID = 'xxxxxxxxxx'
pokemondb = pd.read_csv('Pokemon Database.csv')
pokemon = []
for i in pokemondb['Pokemon Name']:
    i = i.strip('"')
    pokemon.append(i.lower())
pokemon.append('ball')



In [5]:

def get_results(payload):
    try:
        api = Finding(siteid='EBAY-GB', appid=APPLICATION_ID, config_file=None)
        response = api.execute('findItemsAdvanced', payload)
        return response.dict()
    except ConnectionError as e:
        print(e)
        print(e.response.dict())

def search_ebay(payload):
    
    results = get_results(payload)
    if results['searchResult']['_count'] == '0':
        return False
    else:    
    

        df_items = pd.DataFrame(columns=['title', 'viewItemURL', 
                                    'currentPrice'])

        items_list = results['searchResult']['item']
            


        for item in items_list:
            row = {
                'title': item.get('title'),
                'viewItemURL': item.get('viewItemURL'),
                'currentPrice': item.get('sellingStatus').get('currentPrice').get('value')
                
            }

            df_items = df_items.append(row, ignore_index=True)

        return df_items

In [ ]:
psaList = open("input.txt")
stripped_list = []
description_list = []
deadwords = ['pokemon', '\n', 'promo', 'holo', 'movie', 'game', 'tcg','and', 'united', 'kingdom', 'full', 'art', '&']

for i in psaList:
    keywords = []
    raw = i.replace('\t', " ")
    raw = raw.replace('\n', " \n")
    raw = raw.replace("/", " ")
    raw = raw.replace("-", " ")
    refined = (raw.split(" "))
    if refined[3] == "GEM" or refined [3] == "NEAR":
        refined.pop(3)
    if refined[len(refined)-1] == "Card":
        refined.pop(len(refined)-1)
    elif refined[len(refined)-2] == "Card":
        refined.pop(len(refined)-2)
    if refined[4] == "MINT":
        refined.pop(4)
    
    refined2 = []
    for i in refined:
        refined2.append(i)
    description = ""
    for i in range (5, len(refined2)):
        description = description + str(refined2[i]) + " "
    refined2.insert(5, description)
    refined2.pop(0)
    refined2.pop(0)
    refined2.pop(1)
    for i in range (3, len(refined2) - 1):
        refined2.pop(3)
    description_list.append(refined2)
    description = ""
    for i in range (5, len(refined)):
        if refined[i].lower() in deadwords:
            pass
        elif refined[i].lower() in pokemon:
            keywords.append(refined[i])
        elif any(chr.isdigit() for chr in refined[i]) == True:
            if refined[i].isdigit() == True:
                if int(refined[i]) > 1990:
                    description = description + str(refined[i]) + " "
                else:
                    keywords.append(refined[i])
            else:
                keywords.append(refined[i])
            
        else:

            description = description + str(refined[i]) + " "
        
        
    print (description)
    for i in range (5, len(refined) - 1):
        refined.pop(5)
    refined.insert(5, description)
    refined.pop(0)
    refined.pop(0)
    refined.pop(1)
    refined.append(keywords)
    
    
    
    stripped_list.append(refined)



In [ ]:
#stripped_list =stripped_list[0:50]


finalresults = []
minresults=[]
avgresults=[]
for i in stripped_list:
    tempstring = ""
    for x in i[4]:
        tempstring = tempstring + x + " "
    search = tempstring + 'PSA ' + str(i[1])
    search2 = i[2].replace("\n","") 
    search2 = "("+search2+")"
    search2 = search2.replace(" ", ",")
    search = search + " " + search2
    payload = {
        'keywords': search, 
        'categoryId': ['183454'],
        'itemFilter': [
            {'name': 'LocatedIn', 'value': 'GB'},
            {'name': 'ListingType', 'value': 'FixedPrice'}
        ],
        'sortOrder': 'StartTimeNewest',
        }
    df2 = search_ebay(payload)
   
    if isinstance(df2, pd.DataFrame) == False:
        minresults.append('NA')
        avgresults.append('NA')
        finalresults.append('NA')
    else:

        df2 = df2.head(10)
        df2['currentPrice'] = pd.to_numeric(df2['currentPrice'], downcast="float")
        minresults.append((df2['currentPrice'].min()))
        avgresults.append((df2['currentPrice'].mean()))
        finalresults.append(df2)
    

    
   
    



In [ ]:


description_list2 = description_list
count = 0
for i in description_list2:
    if isinstance(finalresults[count], pd.DataFrame) == False:
        pass
    else:
        i.insert(3, str(minresults[count]))

    
    
        temp = finalresults[count].sort_values(by =['currentPrice'])
        temp = temp.head(1)
        temp2 = temp['title'].values.tolist()        
        
        
        newstring = ""
        for chr in temp2[0]:
            if ord(chr) >= 0 and ord(chr) <= 126:
                newstring += chr
        i.insert(4, newstring)
        url = temp['viewItemURL'].values.tolist()  
        i.insert(5, url[0])
        i.insert(6, str(avgresults[count]))
    count = count + 1
print (description_list2)
#print (finalresults)


In [ ]:
#psa grade breakdown
grades = ['1','2','3','4','5','6','7','8','9','10']
for x in grades:
    count = 0
    for i in description_list:
        if i[1] == x:
            count = count + 1
    print (x + " " + str(count))

In [10]:
import csv
fields = ['Cert #', 'Grade', 'Description', 'Lowest BiN', 'Lowest BiN desc', 'Lowest BiN url' 'Average BiN']
with open('output.csv', 'w', newline = '\n') as csvfile:
    my_writer = csv.writer(csvfile)
    my_writer.writerow(fields)
    my_writer.writerows(description_list2)